# Classes
I want to make a data class for all the data i'm loading in. 
This keeps everything clean.

Initialize the class. This will make it cleaner when I want to access things like score or text. (All functions within a class need the 'self' parameter)

I will aslo maka a sentiment function that defines 4-5 stars as positive, 1-2 stars means negative and 3 stars means neutral

The reason for the Sentiment class is so that I dont always have to type NEGATIVE OR POSITIVE. I can just call the class sentiment.
This ensures consistancy in the code.

The ReviewContainer will keep things neat and help me evenly distribute reviews. I will ceate a container class for my reviews so I can call evenly distribute method. That will even out my training data in my test data  

In [115]:
import random

class Sentiment:
    NEGATIVE = "NEGATIVE"
    NEUTRAL = "NEUTRAL"
    POSITIVE = "POSITIVE"

class Review:
    def __init__(self, text, score):
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()
    
    def get_sentiment(self):
        if self.score <= 2:
            return Sentiment.NEGATIVE
        elif self.score == 3:
            return Sentiment.NEUTRAL
        else: # Score of 4 or 5
            return Sentiment.POSITIVE
        
class ReviewContainer:
    def __init__(self, reviews):
        self.reviews = reviews
    
    def get_text(self):
        return [x.text for x in self.reviews]
    
    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]
    # This function looks at all the reviews and maps every sentiment.
    # The list() needs to be included because it needs to turn the results back into a list. 
    def evenly_distribute(self):
        negative = list(filter(lambda x: x.sentiment == Sentiment.NEGATIVE, self.reviews))
        positive = list(filter(lambda x: x.sentiment == Sentiment.POSITIVE, self.reviews))
        positive_shrunk = positive[:len(negative)]
        self.reviews = negative + positive_shrunk
        random.shuffle(self.reviews)

In [116]:
import json

file_name = './data/sentiment/Books_small_10000.json'

#Storing the reviews
reviews = []

# Opening the file and saving the results into the above empty list
with open(file_name) as f:
    for line in f:
        review = json.loads(line)
        reviews.append(Review(review['reviewText'], review['overall']))


In [117]:
# If I wanted just the sentiment i can do
reviews[5].sentiment

'POSITIVE'

In [118]:
# If I wanted just the text i can do
reviews[5].text

'I hoped for Mia to have some peace in this book, but her story is so real and raw.  Broken World was so touching and emotional because you go from Mia\'s trauma to her trying to cope.  I love the way the story displays how there is no "just bouncing back" from being sexually assaulted.  Mia showed us how those demons come for you every day and how sometimes they best you. I was so in the moment with Broken World and hurt with Mia because she was surrounded by people but so alone and I understood her feelings.  I found myself wishing I could give her some of my courage and strength or even just to be there for her.  Thank you Lizzy for putting a great character\'s voice on a strong subject and making it so that other peoples story may be heard through Mia\'s.'

# Bag Of Words (BOW)/Train_Test
When building ML models around text data, things can get funky. ML models like numeric/matricies/vector data types. I want to convert the strings into quantatative vectors.

But before I get to that, I need to train my test and training data. 


In [119]:
from sklearn.model_selection import train_test_split

# I want 33% of the reviews to be used for testing and 66% for training. 
# Random state gives me the same results back every time. 
training, test = train_test_split(reviews, test_size=0.33, random_state=42)

In [120]:
train_container = ReviewContainer(training)
test_container = ReviewContainer(test)

In [121]:
# Prep data:
# Passing the training set into my BOW vectorizer.
# By passing x(text) and y(sentiment)into the model, I want it to return the results.

train_container.evenly_distribute()
train_x = train_container.get_text()
train_y = train_container.get_sentiment()

test_container.evenly_distribute()
test_x = test_container.get_text()
test_y = test_container.get_sentiment()

In [122]:
# Bag of Words vectorization
from sklearn.feature_extraction.text import CountVectorizer

In [123]:
# What I see here as my output is what I want to use as my training input.
# I did two steps in this section, 
vectorizer = CountVectorizer()
train_x_vectors = vectorizer.fit_transform(train_x)

test_x_vectors = vectorizer.transform(test_x)

In [124]:
# To visualize this in a better way, I will print out the above vector contents.
print(train_x_vectors[0])

  (0, 6411)	1
  (0, 2688)	1
  (0, 7976)	1
  (0, 991)	1
  (0, 7819)	1
  (0, 4982)	1
  (0, 7967)	1
  (0, 149)	1
  (0, 7625)	1
  (0, 423)	1
  (0, 3573)	1
  (0, 2230)	1
  (0, 4497)	1
  (0, 8497)	1
  (0, 8409)	1
  (0, 8760)	1
  (0, 1728)	1
  (0, 3264)	1
  (0, 7026)	1
  (0, 5787)	1


In [125]:
train_y

['POSITIVE',
 'NEGATIVE',
 'POSITIVE',
 'NEGATIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'NEGATIVE',
 'NEGATIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'NEGATIVE',
 'NEGATIVE',
 'NEGATIVE',
 'NEGATIVE',
 'NEGATIVE',
 'POSITIVE',
 'NEGATIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'NEGATIVE',
 'POSITIVE',
 'NEGATIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'NEGATIVE',
 'NEGATIVE',
 'POSITIVE',
 'POSITIVE',
 'NEGATIVE',
 'NEGATIVE',
 'POSITIVE',
 'NEGATIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'NEGATIVE',
 'NEGATIVE',
 'NEGATIVE',
 'NEGATIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'NEGATIVE',
 'NEGATIVE',
 'NEGATIVE',
 'POSITIVE',
 'NEGATIVE',
 'POSITIVE',
 'NEGATIVE',
 'POSITIVE',
 'NEGATIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'NEGATIVE',
 'POSITIVE',

# Classification
Sklearn offers a ton of classifiers so the key thing to do is use a couple of different classifiers and test them out.

In [126]:
# I will start with a linear SVM
from sklearn import svm

In [127]:
# This would give me a classifier that I can fit to my x and y. 
# To do this i just pass in the train_x_vectors and train_y into the .fit method
clf_svm = svm.SVC(kernel='linear')

clf_svm.fit(train_x_vectors, train_y)

SVC(kernel='linear')

In [128]:
# Now I can test this by trying to predict whether it is positive or negative
test_x[0]

'Better than the previous book but still kinda cheesy and the plot is rather convoluted. It seemed like the Arthur ran out of stuff to say and tried to rush through to the end. The writing is kind of stiff too'

In [129]:
# Most of these classifiers have a predict method.
# I would just need to pass in the test data
clf_svm.predict(test_x_vectors[0])

array(['NEGATIVE'], dtype='<U8')

In [130]:
# Testing out Decision Tree classifier
from sklearn.tree import DecisionTreeClassifier

clf_dec = DecisionTreeClassifier()
clf_dec.fit(train_x_vectors, train_y)

clf_dec.predict(test_x_vectors[0])

array(['POSITIVE'], dtype='<U8')

In [131]:
# Logistoc Regression
from sklearn.linear_model import LogisticRegression

clf_log = LogisticRegression()
clf_log.fit(train_x_vectors, train_y)

clf_log.predict(test_x_vectors[0])

array(['NEGATIVE'], dtype='<U8')

# Evaluation 
Now that I have some trainined models, I will evealuate them by looking at the entire test set. This will show me how accurate I predicted those test examples.

In [132]:
# Mean Accuracy on all my labels 
clf_svm.score(test_x_vectors, test_y)

0.7980769230769231

In [133]:
clf_dec.score(test_x_vectors, test_y)

0.6322115384615384

In [134]:
clf_log.score(test_x_vectors, test_y)

0.8149038461538461

In [135]:
# F1 scores is something that I am more interested in.
from sklearn.metrics import f1_score

f1_score(test_y, clf_svm.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEGATIVE])

# I can see that this classifier worked great for the positive reviews, but it sucked for the rest. 
# I want to fix this because right now my model is only predicting positive and I want it to be-
# good at prediciting all my sentiment values. 

array([0.8028169 , 0.79310345])

In [136]:
f1_score(test_y, clf_dec.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEGATIVE])

array([0.64335664, 0.62034739])

In [137]:
f1_score(test_y, clf_log.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEGATIVE])

array([0.82051282, 0.808933  ])

In [138]:
# Testing my model 
# My model should recognize that the first review is positive and the other two are negative.
test_set = ['I thoroughly enjoyed this, 5 stars', 'bad book do not buy', 'horriable waste of time']
new_test = vectorizer.transform(test_set)

clf_svm.predict(new_test)

array(['POSITIVE', 'NEGATIVE', 'NEGATIVE'], dtype='<U8')